<a href="https://colab.research.google.com/github/DataSavvyYT/experiments/blob/main/1_llm_finetune/dev/1_gemma_unsloth_w_o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import os

# Update 'python3.x' to match the current Colab version (usually python3.11 or python3.12 in late 2025)
env_path = "/content/drive/MyDrive/unsloth_env/lib/python3.12/site-packages"

if env_path not in sys.path:
    sys.path.append(env_path)

In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [4]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
# 1. Load the model
model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "unsloth/gemma-2-4b", # You can swap this for 'unsloth/mistral-7b'
    model_name="unsloth/gemma-3-4b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.12.8: Fast Gemma3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


In [8]:
# 2. Add LoRA adapters (The "Sticky Notes")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [9]:
DATASET_PATH = "json"
# ----------------------------
# Load dataset
# Expect JSONL with fields: instruction, input, output
# ----------------------------
dataset = load_dataset(
    DATASET_PATH,
    data_files={
        "train": "/content/drive/MyDrive/data/promotion/train.jsonl",
        "validation": "/content/drive/MyDrive/data/promotion/validation.jsonl"
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [10]:
dataset['train'].to_pandas().head()

,instruction,input,output
0,Predict promotion effectiveness.,Campaign: Student Special\nChannel: Facebook A...,effective
1,Predict promotion effectiveness.,Campaign: Monsoon Clearance\nChannel: Instagra...,not effective
2,Predict promotion effectiveness.,Campaign: Weekend Bonanza\nChannel: Email\nBud...,effective
3,Predict promotion effectiveness.,Campaign: Student Special\nChannel: SMS\nBudge...,not effective
4,Predict promotion effectiveness.,Campaign: Clearance Blowout\nChannel: WhatsApp...,not effective


In [11]:
# Formatting function to create supervised prompts
SYSTEM_PREFIX = "You are an analyst that predicts promotion effectiveness based on campaign details."
INSTR_TEMPLATE = """<system>
{system}
</system>
<instruction>
{instruction}
</instruction>
<input>
{inp}
</input>
<output>
{out}
</output>"""

In [12]:
def format_example(example):
    instruction = example.get("instruction", "Predict promotion effectiveness.")
    inp = example.get("input", "")
    out = example.get("output", "")
    # SFTTrainer learns to map input -> output; include output as labels portion
    return INSTR_TEMPLATE.format(system=SYSTEM_PREFIX, instruction=instruction, inp=inp, out=out)

def map_fn(batch):
    texts = [format_example(ex) for ex in batch]
    return {"text": texts}

train_ds = dataset["train"].map(lambda ex: {"text": format_example(ex)})
eval_ds = dataset["validation"].map(lambda ex: {"text": format_example(ex)})

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
train_ds.to_pandas().head()

,instruction,input,output,text
0,Predict promotion effectiveness.,Campaign: Student Special\nChannel: Facebook A...,effective,<system>\nYou are an analyst that predicts pro...
1,Predict promotion effectiveness.,Campaign: Monsoon Clearance\nChannel: Instagra...,not effective,<system>\nYou are an analyst that predicts pro...
2,Predict promotion effectiveness.,Campaign: Weekend Bonanza\nChannel: Email\nBud...,effective,<system>\nYou are an analyst that predicts pro...
3,Predict promotion effectiveness.,Campaign: Student Special\nChannel: SMS\nBudge...,not effective,<system>\nYou are an analyst that predicts pro...
4,Predict promotion effectiveness.,Campaign: Clearance Blowout\nChannel: WhatsApp...,not effective,<system>\nYou are an analyst that predicts pro...


In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set this to 60 for a quick demo, or 1000 for real training
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()


Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/400 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 400 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,788,480 of 4,332,867,952 (0.76% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,3.494200
2,3.528500
3,3.257600
4,2.817700
5,2.339800
6,1.886600
7,1.692200
8,1.360100
9,1.118900
10,0.966100


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▆▅▃▃▅▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▄▇███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▁▁
train/loss,██▇▆▅▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,1274004362641920.0
train/epoch,1.2
train/global_step,60
train/grad_norm,0.44015
train/learning_rate,0.0
train/loss,0.2505


TrainOutput(global_step=60, training_loss=0.6392137564718723, metrics={'train_runtime': 276.8236, 'train_samples_per_second': 1.734, 'train_steps_per_second': 0.217, 'total_flos': 1274004362641920.0, 'train_loss': 0.6392137564718723, 'epoch': 1.2})

In [30]:
def predict_effectiveness(description: str) -> str:
    prompt = f"""<system>
{SYSTEM_PREFIX}
</system>
<instruction>
Predict promotion effectiveness as one of: "effective", "not effective", or a probability between 0 and 1.
</instruction>
<input>
{description}
</input>
<output>
"""
    inputs = tokenizer(text=prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # naive parse: take last line after <output>
    return text.split("<output>")[-1].strip()

In [25]:
print(predict_effectiveness("Campaign: Diwali Sale; Channel: Email; Budget: 5 Lakh INR; Audience: Returning; Discount: 10%; Duration: 5 days; Past CTR: 2.8%"))


effective
</output>
<instruction>
Predict promotion effectiveness based on campaign details.
</


In [31]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

true_labels = []
predictions = []

print("Starting evaluation on validation dataset...")
for i, example in enumerate(eval_ds):
    if i % 10 == 0:
        print(f"Processing example {i}/{len(eval_ds)}")

    campaign_description = example["input"]
    true_label = example["output"].strip().lower()

    # Get prediction from the model
    predicted_raw = predict_effectiveness(campaign_description)
    if predicted_raw.startswith("not effective"):
      final_prediction = "not effective"
    elif predicted_raw.startswith("effective"):
      final_prediction = "effective"

    predicted_text = predicted_raw.strip().lower()

    predicted_label = predicted_text
    try:
        # Check if the output is a probability
        predicted_prob = float(predicted_text)
        # Assuming a threshold of 0.5 for classification if a probability is returned
        predicted_label = "effective" if predicted_prob > 0.5 else "not effective"
    except ValueError:
        # If not a probability, assume it's directly the label string
        predicted_label = predicted_text

    true_labels.append(true_label)
    predictions.append(predicted_label)

print("\nEvaluation complete.")

# Ensure we have labels for classification_report to avoid errors
unique_labels = sorted(list(set(true_labels + predictions)))

if not true_labels:
    print("No true labels found in the validation dataset. Cannot compute metrics.")
elif not unique_labels:
    print("No labels (true or predicted) found. Cannot compute metrics.")
elif len(unique_labels) == 1:
    # If only one class is present in true or predicted labels, accuracy_score works,
    # but classification_report might struggle. Handle gracefully.
    print(f"Only one label ('{unique_labels[0]}') present in true or predicted labels.")
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {accuracy:.4f}")
    # No classification report if only one label is present
else:
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {accuracy:.4f}")

    print("\nClassification Report:")
    # Use zero_division=0 to prevent warnings/errors if a class has no samples/predictions
    print(classification_report(true_labels, predictions, labels=unique_labels, zero_division=0))

Starting evaluation on validation dataset...
Processing example 0/100
Processing example 10/100
Processing example 20/100
Processing example 30/100
Processing example 40/100
Processing example 50/100
Processing example 60/100
Processing example 70/100
Processing example 80/100
Processing example 90/100

Evaluation complete.
Overall Accuracy: 0.0000

Classification Report:
                                                                                                  precision    recall  f1-score   support

                                                                                       effective       0.00      0.00      0.00      59.0
 effective
</output>
<instruction>
predict promotion effectiveness based on campaign details.
</       0.00      0.00      0.00       0.0
                                                                                   not effective       0.00      0.00      0.00      41.0
not effective
</output>
<instruction>
predict promotion effectiveness ba

In [35]:
def predict_effectiveness(description: str) -> str:
    print("prmotion : "+description)
    prompt = f"""<system>
{SYSTEM_PREFIX}
</system>
<instruction>
Predict promotion effectiveness as one of: "effective", "not effective", or a probability between 0 and 1.
</instruction>
<input>
{description}
</input>
<output>
"""
    inputs = tokenizer(text=prompt, return_tensors="pt").to(model.device)
    input_ids_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        generated_output = model.generate(**inputs, max_new_tokens=20, do_sample=False)

    generated_text_ids = generated_output[0][input_ids_len:]
    decoded_text_raw = tokenizer.decode(generated_text_ids, skip_special_tokens=True)
    decoded_text_clean = decoded_text_raw.strip().lower()

    # Debug print the raw decoded text to understand model's output
    #print(f"DEBUG_PREDICT: Decoded raw output: '{decoded_text_raw}'")
    #print(f"DEBUG_PREDICT: Cleaned for parsing: '{decoded_text_clean}'")

    final_prediction = ""

    # Strategy 1: Look for exact phrases "effective" or "not effective" at the beginning
    if decoded_text_clean.startswith("not effective"):
        final_prediction = "not effective"
    elif decoded_text_clean.startswith("effective"):
        final_prediction = "effective"
    else:
        # Strategy 2: If model emits </output>, try to parse content before it
        if "</output>" in decoded_text_clean:
            content_before_tag = decoded_text_clean.split("</output>")[0].strip()
            if content_before_tag.startswith("not effective"):
                final_prediction = "not effective"
            elif content_before_tag.startswith("effective"):
                final_prediction = "effective"
            elif content_before_tag.startswith("0") or content_before_tag.startswith("1") or "." in content_before_tag:
                 # Strategy 3: Try to parse as probability if it seems like one
                 try:
                     prob = float(content_before_tag.split()[0])
                     final_prediction = "effective" if prob > 0.5 else "not effective"
                 except (ValueError, IndexError):
                     pass # Not a clean probability
            else:
                # If still no clear label, take the first line of what was before </output>
                final_prediction = content_before_tag.split('\n')[0].strip()
        else:
            # Strategy 4: No </output> tag, try to parse the first line
            first_line = decoded_text_clean.split('\n')[0].strip()
            if first_line.startswith("not effective"):
                final_prediction = "not effective"
            elif first_line.startswith("effective"):
                final_prediction = "effective"
            elif first_line.startswith("0") or first_line.startswith("1") or "." in first_line:
                 # Strategy 5: Try to parse as probability from first line
                 try:
                     prob = float(first_line.split()[0])
                     final_prediction = "effective" if prob > 0.5 else "not effective"
                 except (ValueError, IndexError):
                     pass
            else:
                final_prediction = first_line # Last resort, return the first line

    # If final_prediction is still empty after all strategies, assign the most reasonable fallback
    if not final_prediction:
        final_prediction = decoded_text_clean.split('\n')[0].strip() # Take first line as absolute fallback

    print(f"DEBUG_PREDICT: Final parsed prediction: '{final_prediction}'")
    return final_prediction


In [36]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

true_labels = []
predictions = []

print("Starting evaluation on validation dataset...")
for i, example in enumerate(eval_ds):
    if i % 10 == 0:
        print(f"Processing example {i}/{len(eval_ds)}")

    campaign_description = example["input"]
    true_label = example["output"].strip().lower()

    # Get prediction from the model
    predicted_raw = predict_effectiveness(campaign_description)
    predicted_text = predicted_raw.strip().lower()

    predicted_label = ""
    try:
        # Check if the output is a probability
        predicted_prob = float(predicted_text)
        # Assuming a threshold of 0.5 for classification if a probability is returned
        predicted_label = "effective" if predicted_prob > 0.5 else "not effective"
    except ValueError:
        # If not a probability, assume it's directly the label string
        predicted_label = predicted_text

    true_labels.append(true_label)
    predictions.append(predicted_label)

print("\nEvaluation complete.")

# Ensure we have labels for classification_report to avoid errors
unique_labels = sorted(list(set(true_labels + predictions)))

if not true_labels:
    print("No true labels found in the validation dataset. Cannot compute metrics.")
elif not unique_labels:
    print("No labels (true or predicted) found. Cannot compute metrics.")
elif len(unique_labels) == 1:
    # If only one class is present in true or predicted labels, accuracy_score works,
    # but classification_report might struggle. Handle gracefully.
    print(f"Only one label ('{unique_labels[0]}') present in true or predicted labels.")
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {accuracy:.4f}")
    # No classification report if only one label is present
else:
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {accuracy:.4f}")

    print("\nClassification Report:")
    # Use zero_division=0 to prevent warnings/errors if a class has no samples/predictions
    print(classification_report(true_labels, predictions, labels=unique_labels, zero_division=0))

Starting evaluation on validation dataset...
Processing example 0/100
prmotion : Campaign: New Arrival Launch
Channel: In-App Banner
Budget: 18 Lakh INR
Audience: Credit Card Holders
Discount: Buy 1 Get 1
Duration: 7 days
Past CTR: 3.32%
Avg AOV: 12000 INR
Category: Home Decor
DEBUG_PREDICT: Final parsed prediction: 'effective'
prmotion : Campaign: Cyber Monday
Channel: In-App Banner
Budget: 10 Lakh INR
Audience: Parents
Discount: 10%
Duration: 6 days
Past CTR: 4.26%
Avg AOV: 800 INR
Category: Groceries
DEBUG_PREDICT: Final parsed prediction: 'effective'
prmotion : Campaign: Freedom Sale
Channel: Influencer Collab
Budget: 10 Lakh INR
Audience: Gen Z (18-24)
Discount: 5%
Duration: 14 days
Past CTR: 3.29%
Avg AOV: 12000 INR
Category: Footwear
DEBUG_PREDICT: Final parsed prediction: 'not effective'
prmotion : Campaign: New Arrival Launch
Channel: SMS
Budget: 20 Lakh INR
Audience: Credit Card Holders
Discount: 20%
Duration: 2 days
Past CTR: 1.91%
Avg AOV: 12000 INR
Category: Electronics
DE

Here is the classification report and interpretation

### Classification Report

| Class | Precision | Recall | F1-Score | Support |
| --- | --- | --- | --- | --- |
| **effective** | 0.60 | 0.95 | 0.73 | 59 |
| **not effective** | 0.50 | 0.07 | 0.13 | 41 |
| **Accuracy** |  |  | **0.59** | **100** |
| **Macro Avg** | 0.55 | 0.51 | 0.43 | 100 |
| **Weighted Avg** | 0.56 | 0.59 | 0.48 | 100 |

---

### Interpretation of Results

* **Overall Accuracy (59.00%):** This means the model correctly predicted the effectiveness for 59 out of 100 promotions in the validation set.

#### 'Effective' Class

* **Precision (0.60):** When the model predicts a promotion is 'effective', it is correct 60% of the time.
* **Recall (0.95):** The model correctly identifies 95% of all truly 'effective' promotions.
* **F1-Score (0.73):** This is the harmonic mean of precision and recall, indicating a reasonably good balance for this class.

#### 'Not Effective' Class

* **Precision (0.50):** When the model predicts a promotion is 'not effective', it is correct 50% of the time.
* **Recall (0.07):** The model is very poor at identifying truly 'not effective' promotions, catching only 7% of them. This suggests it frequently misclassifies 'not effective' promotions as 'effective'.
* **F1-Score (0.13):** A very low F1-score indicates poor performance for this class.

### Summary

The model shows a strong bias towards predicting promotions as 'effective'. While it correctly identifies most 'effective' promotions (high recall for 'effective'), it struggles significantly with identifying 'not effective' promotions. This imbalance in performance might be due to the dataset (fewer 'not effective' examples or less distinct features) or the model's training. Further fine-tuning or data balancing might be needed to improve performance on the 'not effective' class.

> I have successfully evaluated the model's accuracy on the validation dataset and provided a detailed breakdown of the results, including overall accuracy, precision, recall, and F1-score for both 'effective' and 'not effective' classes. The analysis indicated a bias towards predicting 'effective' promotions. If you have any further questions or tasks, please let me know!

### Next Step

Would you like me to suggest techniques for balancing your dataset (like SMOTE or class weighting) to improve the "not effective" class performance?